In [1]:
import pandas as pd
import numpy as np
import math
from nltk.stem.snowball import SnowballStemmer
from itertools import islice
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from collections import defaultdict
from sklearn.svm import LinearSVC

In [2]:
data = pd.read_csv("lyrics.csv")

In [3]:
data = data.replace({'[iI][Nn][sS][Tt][Rr][Uu][Mm][Ee][Nn][Tt]' : math.nan}, regex=True)
data = data.replace({'instrument': math.nan})
data = data.replace({'Instrument' : math.nan})
data = data.dropna()
data = data[data.genre != "Not Available"]
data = data.replace({'\n': ' '}, regex=True)

In [4]:
len(data)

237769

In [5]:
data = data.loc[(data['genre'] == 'Pop') | (data['genre'] == 'Rock') | (data['genre'] == 'Hip-Hop') | (data['genre'] == 'Metal') | (data['genre'] == 'Country')]
#x = training_genre_lyrics.loc[training_genre_lyrics['genre'] == 'Jazz']
len(data)
data.head()

,index,song,year,artist,genre,lyrics
0,0,ego-remix,2009,beyonce-knowles,Pop,"Oh baby, how you doing? You know I'm gonna cut..."
1,1,then-tell-me,2009,beyonce-knowles,Pop,"playin' everything so easy, it's like you seem..."
2,2,honesty,2009,beyonce-knowles,Pop,If you search For tenderness It isn't hard to ...
3,3,you-are-my-rock,2009,beyonce-knowles,Pop,"Oh oh oh I, oh oh oh I [Verse 1:] If I wrote a..."
4,4,black-culture,2009,beyonce-knowles,Pop,"Party the people, the people the party it's po..."


In [6]:
data_pop = data.loc[(data['genre'] == 'Pop')]
data_rock = data.loc[(data['genre'] == 'Rock')]
data_hiphop = data.loc[(data['genre'] == "Hip-Hop")]
data_metal = data.loc[(data['genre']== "Metal")]
data_country = data.loc[(data['genre'] == "Country")]


def get_sample(num_each):
    pop_df = data_pop.sample(n = num_each, replace = True)
    rock_df = (data_rock.sample(n = num_each, replace=True))
    hiphop_df = (data_hiphop.sample(n = num_each, replace=True))
    metal_df = (data_metal.sample(n = num_each, replace=True))
    country_df = (data_country.sample(n = num_each, replace=True))
    ret_df = pd.concat([pop_df, rock_df, hiphop_df, metal_df, country_df])
    ret_df = ret_df.sample(frac=1)
    return ret_df

training_set = get_sample(10000)
validation_set = get_sample(10000)
test_set = get_sample(10000)

#training_50k = data.sample(n=50000, replace=True)
#validation_50k = data.sample(n=50000, replace=True)
#testing_50k = data.sample(n=50000, replace=True)



In [8]:
#training_test['genre'].value_counts()

In [9]:
# Filter just genre and lyrics columns
training_genre_lyrics = training_set.filter([ 'genre', 'lyrics'])
validation_genre_lyrics = validation_set.filter(['genre', 'lyrics'])
testing_genre_lyrics = test_set.filter(['genre', 'lyrics'])

In [10]:
#goes thru training set and append genre pop's lyrics
pop_lyrics = []
for x in training_genre_lyrics.iterrows():
    if x[1][0] == 'Pop':
        pop_lyrics.append(x[1][1])
pop_lyrics = pd.DataFrame({"ly": pop_lyrics})
print (len(pop_lyrics))

10000


In [11]:
rock_lyrics = []
for x in training_genre_lyrics.iterrows():
    if x[1][0] == 'Rock':
        rock_lyrics.append(x[1][1])
print (len(rock_lyrics))
rock_lyrics = pd.DataFrame({"ly": rock_lyrics})

10000


In [12]:
hiphop_lyrics = []
for x in training_genre_lyrics.iterrows():
    if x[1][0] == 'Hip-Hop':
        hiphop_lyrics.append(x[1][1])
hiphop_lyrics = pd.DataFrame({"ly": hiphop_lyrics})
print (len(hiphop_lyrics))

10000


In [13]:
metal_lyrics = []
for x in training_genre_lyrics.iterrows():
    if x[1][0] == 'Metal':
        metal_lyrics.append(x[1][1])
metal_lyrics = pd.DataFrame({"ly": metal_lyrics})
print (len(metal_lyrics))

10000


In [14]:
country_lyrics = []
for x in training_genre_lyrics.iterrows():
    if x[1][0] == 'Country':
        country_lyrics.append(x[1][1])
country_lyrics = pd.DataFrame({"ly": country_lyrics})
print (len(country_lyrics))

10000


In [96]:
ENGLISH_STOP_WORDS = frozenset([
    "a", "about", "above", "across", "after", "afterwards", "again", "against",
    "all", "almost", "alone", "along", "already", "also", "although", "always",
    "am", "among", "amongst", "amoungst", "amount", "an", "and", "another",
    "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are",
    "around", "as", "at", "back", "be", "became", "because", "become",
    "becomes", "becoming", "been", "before", "beforehand", "behind", "being",
    "below", "beside", "besides", "between", "beyond", "bill", "both",
    "bottom", "but", "by", "call", "can", "cannot", "cant", "co", "con",
    "could", "couldnt", "cry", "de", "describe", "detail", "do", "done",
    "down", "due", "during", "each", "eg", "eight", "either", "eleven", "else",
    "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone",
    "everything", "everywhere", "except", "few", "fifteen", "fifty", "fill",
    "find", "fire", "first", "five", "for", "former", "formerly", "forty",
    "found", "four", "from", "front", "full", "further", "get", "give", "go",
    "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter",
    "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his",
    "how", "however", "hundred", "i", "ie", "if", "in", "inc", "indeed",
    "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter",
    "latterly", "least", "less", "ltd", "made", "many", "may", "me",
    "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly",
    "move", "much", "must", "my", "myself", "name", "namely", "neither",
    "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone",
    "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on",
    "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our",
    "ours", "ourselves", "out", "over", "own", "part", "per", "perhaps",
    "please", "put", "rather", "re", "same", "see", "seem", "seemed",
    "seeming", "seems", "serious", "several", "she", "should", "show", "side",
    "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone",
    "something", "sometime", "sometimes", "somewhere", "still", "such",
    "system", "take", "ten", "than", "that", "the", "their", "them",
    "themselves", "then", "thence", "there", "thereafter", "thereby",
    "therefore", "therein", "thereupon", "these", "they", "thick", "thin",
    "third", "this", "those", "though", "three", "through", "throughout",
    "thru", "thus", "to", "together", "too", "top", "toward", "towards",
    "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us",
    "very", "via", "was", "we", "well", "were", "what", "whatever", "when",
    "whence", "whenever", "where", "whereafter", "whereas", "whereby",
    "wherein", "whereupon", "wherever", "whether", "which", "while", "whither",
    "who", "whoever", "whole", "whom", "whose", "why", "will", "with",
    "within", "without", "would", "yet", "you", "your", "yours", "yourself",
    "yourselves"])

In [97]:
# Takes a dataframe/list containing lyrics and returns a tuple with the terms and weights 
def vectorize_feature(genre_lyrics, num_words):
    stemmer = SnowballStemmer("english")
    genre_lyrics['stemmed'] = genre_lyrics.ly.map(lambda x: ' '.join([stemmer.stem(y) for y in x.split(' ')]))
    #genre_lyrics.stemmed.head()
    
    cvec = CountVectorizer(min_df=1,max_df= 0.5, ngram_range=(1,3), stop_words= ENGLISH_STOP_WORDS)
    cvec.fit(genre_lyrics.stemmed)
    
    # -- use for debugging -- x = list(islice(cvec.vocabulary_.items(), 20))
    
    cvec_counts = cvec.transform(genre_lyrics.stemmed)
    
    transformer = TfidfTransformer()
    transformed_weights = transformer.fit_transform(cvec_counts)
    
    weights = np.asarray(transformed_weights.mean(axis=0)).ravel().tolist()
    weights_df = pd.DataFrame({'term': cvec.get_feature_names(), 'weight': weights})
    weights_df = weights_df.sort_values(by='weight', ascending=False)[:num_words]
    
    #term_weight_list = []
    term_weight_dict = defaultdict(int)
    for _, row in weights_df.iterrows():
        # Returns a list of (term, weight) tuples
        #term_phrase_list.append((row['term'], row['weight']))
        term_weight_dict[row['term']] = row['weight']
    
    # return term_weight_list
    return term_weight_dict

In [98]:
rock_dicts = vectorize_feature(rock_lyrics, 1000)
pop_dicts = vectorize_feature(pop_lyrics, 1000)
hiphop_dicts = vectorize_feature(hiphop_lyrics, 1000)
metal_dicts = vectorize_feature(metal_lyrics, 1000)
country_dicts = vectorize_feature(country_lyrics, 1000)

In [99]:
print (len(rock_dicts))

1000


In [95]:
# counter = 0
# rock = list(rock_dicts.keys())
# pop = list(pop_dicts.keys())
# for i in range(1000):
#     if rock[i] == pop[i]:
#         counter += 1

In [100]:
# Given new lyrics, return list of sums of weights
def find_weights(lyrics):
    weight_list = [0]*5
    stemmer = SnowballStemmer("english")
    stemmed_lyrics = ' '.join([stemmer.stem(y) for y in lyrics.split(' ')])
    for i in stemmed_lyrics.split():
        weight_list[0] += rock_dicts[i]
        weight_list[1] += pop_dicts[i]
        weight_list[2] += hiphop_dicts[i]
        weight_list[3] += metal_dicts[i]
        weight_list[4] += country_dicts[i]
    return weight_list.index(max (weight_list))

In [72]:
#find_weights(pop_lyrics.ly[2])

In [101]:
genre = {
    'Rock': 0,
    'Pop' : 1,
    'Hip-Hop' : 2,
    'Metal' : 3,
    'Country' : 4,
}

In [102]:
x_train = [find_weights(x) for x in training_genre_lyrics.lyrics]
#print (x_train)

In [103]:
y_train = [genre[x] for x in training_genre_lyrics.genre]

In [104]:
acc = [(x == y) for (x,y) in zip(x_train, y_train)]
# acc = sum(acc) * 1.0/50000
# print (acc)
counter = 0
for i in range(len(acc)):
    if acc[i] == True:
        counter += 1
print (counter/50000)

0.41436


In [105]:
x_valid = [find_weights(x) for x in validation_genre_lyrics.lyrics]
y_valid = [genre[x] for x in validation_genre_lyrics.genre]

In [106]:
from sklearn.svm import LinearSVC
from sklearn import svm
x_train = np.asarray(x_train).reshape(-1, 1)
x_valid = np.asarray(x_valid).reshape(-1, 1)
bestAcc = 0
bestCLF = None
for c in 0.01, 0.1, 1, 10, 100:
  clf = svm.LinearSVC(C = c) # Linear SVM is faster
  clf.fit(x_train, y_train)
  predictions = [int(x) for x in clf.predict(x_valid)]
  acc = [(x == y) for (x,y) in zip(predictions, y_valid)]
  acc = sum(acc) * 1.0 / len(acc)
  if acc > bestAcc:
    bestAcc = acc
    bestCLF = clf
  print("C = " + str(c) + ": validation accuracy = " + str(acc))

C = 0.01: validation accuracy = 0.28678
C = 0.1: validation accuracy = 0.28678
C = 1: validation accuracy = 0.28678
C = 10: validation accuracy = 0.28678
C = 100: validation accuracy = 0.24404
